## Downloading data

In [ ]:
#uploading file to google colab

from google.colab import files
data_to_load = files.upload()

Saving archive.zip to archive (1).zip


In [ ]:
!unzip -q archive.zip 
!ls

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(r'flipkart_com-ecommerce_sample.csv')

In [5]:
df.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [ ]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [ ]:
df.isna().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

## Data preprocessing

In [ ]:
# taking only required columns
to_drop = ['uniq_id', 'crawl_timestamp', 'product_url', 'pid', 
           'retail_price', 'discounted_price', 'image', 'is_FK_Advantage_product', 
           'product_rating', 'overall_rating', 'product_specifications']
df = df.drop(to_drop, axis=1)
df.head()

,product_name,product_category_tree,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Sicons


In [ ]:
df.product_name.duplicated().sum()

7324

In [ ]:
# removing duplicates
df.drop_duplicates(subset='product_name', keep='first', inplace=True)
df.shape

(12676, 4)

In [ ]:
df_copy = df.copy()
df_copy.head()

,product_name,product_category_tree,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,AW
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Sicons
5,Eternal Gandhi Super Series Crystal Paper Weig...,"[""Eternal Gandhi Super Series Crystal Paper We...",Key Features of Eternal Gandhi Super Series Cr...,Eternal Gandhi


## Preparing data

In [ ]:
df.description = df.description.fillna('')

In [ ]:
df.description.isna().sum()

0

In [ ]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [ ]:
# function to process text
def process_text(input_text):
    tokens = word_tokenize(input_text)
    no_stop = [w for w in tokens if w not in stop_words]
    no_punc = [ch for ch in no_stop if ch.isalpha()]
    stemmed = [(porter.stem(w)) for w in no_punc]
    processed_text = ' '.join(w for w in stemmed)
    return processed_text

In [ ]:
# applying function
df['product_name'] = df['product_name'].apply(process_text)
df['product_category_tree'] = df['product_category_tree'].apply(process_text)
df['description'] = df['description'].apply(process_text)
df['brand'] = df['brand'].astype('str').apply(process_text)

In [ ]:
df.head(10)

,product_name,product_category_tree,description,brand
0,alisha solid women cycl short,cloth women cloth lingeri sleep swimwear short...,key featur alisha solid women cycl short cotto...,alisha
1,fabhomedecor fabric doubl sofa bed,furnitur live room furnitur sofa bed futon fab...,fabhomedecor fabric doubl sofa bed finish colo...,fabhomedecor
2,AW belli,footwear women footwear ballerina AW belli,key featur AW belli sandal wedg heel casual AW...,AW
4,sicon all purpos arnica dog shampoo,pet suppli groom skin coat care shampoo sicon ...,specif sicon all purpos arnica dog shampoo ml ...,sicon
5,etern gandhi super seri crystal paper weight s...,etern gandhi super seri crystal paper weight,key featur etern gandhi super seri crystal pap...,etern gandhi
8,dilli bazaaar belli corpor casual casual,footwear women footwear ballerina dilli bazaaa...,key featur dilli bazaaar belli corpor casual c...,dilli bazaaar
10,ladela belli,footwear women footwear ballerina ladela belli,key featur ladela belli brand ladela color bla...,ladela
11,carrel print women,cloth women cloth sport gym wear swimsuit carr...,key featur carrel print women fabric swimlycra...,carrel
12,sicon all purpos tea tree dog shampoo,pet suppli groom skin coat care shampoo sicon ...,specif sicon all purpos tea tree dog shampoo m...,sicon
14,freelanc vacuum bottl ml bottl,pen stationeri school suppli water bottl freel...,specif freelanc vacuum bottl ml bottl pack gre...,freelanc


In [ ]:
# combining features
df['combined_features'] = df.product_name + df.product_category_tree + df.description + df.brand
df.head()

,product_name,product_category_tree,description,brand,combined_features
0,alisha solid women cycl short,cloth women cloth lingeri sleep swimwear short...,key featur alisha solid women cycl short cotto...,alisha,alisha solid women cycl shortcloth women cloth...
1,fabhomedecor fabric doubl sofa bed,furnitur live room furnitur sofa bed futon fab...,fabhomedecor fabric doubl sofa bed finish colo...,fabhomedecor,fabhomedecor fabric doubl sofa bedfurnitur liv...
2,AW belli,footwear women footwear ballerina AW belli,key featur AW belli sandal wedg heel casual AW...,AW,AW bellifootwear women footwear ballerina AW b...
4,sicon all purpos arnica dog shampoo,pet suppli groom skin coat care shampoo sicon ...,specif sicon all purpos arnica dog shampoo ml ...,sicon,sicon all purpos arnica dog shampoopet suppli ...
5,etern gandhi super seri crystal paper weight s...,etern gandhi super seri crystal paper weight,key featur etern gandhi super seri crystal pap...,etern gandhi,etern gandhi super seri crystal paper weight s...


## Feature extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = 'word', ngram_range=(1, 2), min_df=0)
extracted_features = vectorizer.fit_transform(df['combined_features'])

In [ ]:
extracted_features

<12676x180470 sparse matrix of type '<class 'numpy.float64'>'
	with 1068965 stored elements in Compressed Sparse Row format>

## Using cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(extracted_features, extracted_features)

In [ ]:
cos_sim

array([[1.00000000e+00, 1.16204169e-02, 2.56960689e-02, ...,
        1.63059801e-02, 2.47601741e-02, 1.37860623e-03],
       [1.16204169e-02, 1.00000000e+00, 4.84962489e-02, ...,
        2.96901440e-03, 4.50836520e-03, 3.40213126e-02],
       [2.56960689e-02, 4.84962489e-02, 1.00000000e+00, ...,
        2.41914299e-02, 3.73511592e-02, 1.64027022e-02],
       ...,
       [1.63059801e-02, 2.96901440e-03, 2.41914299e-02, ...,
        1.00000000e+00, 9.12858385e-02, 1.55791239e-04],
       [2.47601741e-02, 4.50836520e-03, 3.73511592e-02, ...,
        9.12858385e-02, 1.00000000e+00, 2.36564633e-04],
       [1.37860623e-03, 3.40213126e-02, 1.64027022e-02, ...,
        1.55791239e-04, 2.36564633e-04, 1.00000000e+00]])

## Getting Recommendations

In [ ]:
df = df.reset_index()

In [ ]:
products = df_copy['product_name']
indices = pd.Series(df.index, index=df_copy['product_name'])
indices

product_name
Alisha Solid Women's Cycling Shorts                                          0
FabHomeDecor Fabric Double Sofa Bed                                          1
AW Bellies                                                                   2
Sicons All Purpose Arnica Dog Shampoo                                        3
Eternal Gandhi Super Series Crystal Paper Weights  with Silver Finish        4
                                                                         ...  
Purple Women Heels                                                       12671
Uberlyfe Large Vinyl Sticker                                             12672
We Witches Comfy Hues Women Wedges                                       12673
Stylistry Women Heels                                                    12674
Uberlyfe Extra Large Vinyl Sticker                                       12675
Length: 12676, dtype: int64

In [ ]:
# function to get top 10 recommendations

def get_recommendations(product):
  ind = indices[product]
  sim_scores = list(enumerate(cos_sim[ind]))
  sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse=True)
  product_indices = [i[0] for i in sim_scores[1:11]]
  return products.iloc[product_indices]

In [ ]:
get_recommendations('Purple Women Heels')

10046          TEN Women Heels
753           Nell Women Heels
10016         Liza Women Heels
10004        Kielz Women Heels
11520       Selfie Women Heels
10022      FatDuck Women Heels
10052     Chicopee Women Heels
9889        Rialto Women Heels
19962    Stylistry Women Heels
10050          iLO Women Heels
Name: product_name, dtype: object

In [ ]:
get_recommendations('Uberlyfe Extra Large Vinyl Sticker')

10248         Happy walls Extra Large PVC vinyl Sticker
19948                      Uberlyfe Large Vinyl Sticker
19822    Oren Empower Extra Large Self Adhesive Sticker
7596               DeStudio Large Wall Stickers Sticker
10265                   Ddreamz Small PVC Vinyl Sticker
16428              Wallmantra Extra Large Vinyl Sticker
15882               DeStudio Small WALL STICKER Sticker
19077            Wall Decal Small Self Adhesive Sticker
19078           Wall Decal Medium Self Adhesive Sticker
15647                 DeStudio Extra Large Wall Sticker
Name: product_name, dtype: object